# Testing Classes to represent assets - NestSmart -

In [1]:
from icecream import ic

In [2]:
import numpy as np
import xarray as xr

class Asset:

    # Will probably need a list of dates for which the levels, payments, etc. correspond (Check pendulum)

    balance_bop = None
    returns = None 
    inflows = None
    outflows = None
    balance_eop = None
    
    def __init__(self,initial_value,growth):
        self.value = xr.DataArray(np.ones(len(growth)+1)) * initial_value


In [ ]:
a = Asset(10,[1,1,1])

In [ ]:
a.value

In [3]:
initial_year = 2000
years_to_simulate = 3
currency = 'GBP'

years = np.arange(initial_year,initial_year+years_to_simulate+1)

initial_balance =100

In [16]:
return_pct = xr.DataArray([np.nan,0.1,0.1,0.1], 
                           coords={"year": years},
                           attrs = {'long_name':'Investment return','units':'%'}
                          )
inflows = xr.DataArray([np.nan,10.0,0.0,0.0], 
                           coords={"year": years},
                           attrs = {'long_name':'Cash inflows','units':currency}
                          )
outflows = xr.DataArray([np.nan,-1.0,-1.0,-0.0], 
                           coords={"year": years},
                           attrs = {'long_name':'Cash outflows','units':currency}
                          )

# To define in or outflows as an absolute figure it will be necesary to convert the absolute to pct using the previous year eop (same as returns are % of previous year eop)


In [7]:
#Change variable names to reflect "starting balance"
balance_eop_tmp = xr.DataArray(np.ones(len(years)) * initial_balance, 
                           coords={"year": years[0:]},
                           attrs = {'long_name':'Balance at end of period','units':currency}
                          )
balance_eop_tmp = balance_eop_tmp * ((return_pct + 1).cumprod())
investment_returns = balance_eop_tmp.diff('year')

In [17]:
return_inflows_tmp = xr.DataArray(np.zeros(len(years)), 
                                  coords={"year": years},
                                  attrs = {'long_name':'Balance at end of period','units':currency}
                                 )
#ic(return_inflows_tmp)

for (i,year) in enumerate(years[1:],start=1):
    years_left = len(years) - i
    tmp = xr.DataArray(np.ones(years_left) * inflows.loc[year].values, 
                       coords={"year": years[i:]},
                       attrs = {'long_name':'Balance at end of period','units':currency}
                      )
    #ic(tmp)
    
    with xr.set_options(arithmetic_join="outer"):
        tmp = tmp * ((return_pct + 1).cumprod())
        #ic(tmp.fillna(0))
        return_inflows_tmp = return_inflows_tmp + tmp.fillna(0)
        
    #ic(return_inflows_tmp)

In [18]:
return_outflows_tmp = xr.DataArray(np.zeros(len(years)), 
                                  coords={"year": years},
                                  attrs = {'long_name':'Balance at end of period','units':currency}
                                 )
#ic(return_inflows_tmp)

for (i,year) in enumerate(years[1:],start=1):
    years_left = len(years) - i
    tmp = xr.DataArray(np.ones(years_left) * outflows.loc[year].values, 
                       coords={"year": years[i:]},
                       attrs = {'long_name':'Balance at end of period','units':currency}
                      )
    #ic(tmp)
    
    with xr.set_options(arithmetic_join="outer"):
        tmp = tmp * ((return_pct + 1).cumprod())
        #ic(tmp.fillna(0))
        return_outflows_tmp = return_outflows_tmp + tmp.fillna(0)
        
    #ic(return_inflows_tmp)

In [20]:
balance_eop_tmp + return_inflows_tmp + return_outflows_tmp

<xarray.DataArray (year: 4)>
array([100.   , 119.9  , 130.68 , 143.748])
Coordinates:
  * year     (year) int64 2000 2001 2002 2003

In [ ]:

#balance_eop_tmp = balance_eop_tmp * ((return_pct + 1).cumprod())
#investment_returns = balance_eop_tmp.diff('year')

# NOW IT NEEDS TO CALCULATE THE INVESTMENT RETURNS EFFECT FROM INFLOWS AND OUTFLOWS

# The final balance eop = original balance + return from orig. bal. + inflows + return from inflows + outflows + ret. from outflows

In [ ]:

return_inflows_tmp

In [ ]:
change_abs = investment_returns + inflows + outflows

balance_eop = xr.DataArray(np.ones(4) * initial_balance, 
                           coords={"year": years[0:]},
                           attrs = {'long_name':'Balance at end of period','units':currency}
                          )
balance_eop = balance_eop + change_abs


In [ ]:
balance_eop

In [ ]:
total_change_pct = return_pct + inflows_pct + outflows_pct
total_change_pct.attrs['long_name'] = 'Total change in period'
total_change_pct.attrs['units'] = '%'

In [ ]:
total_change_pct

In [ ]:
# Next steps
#     - Convert total change to growth factor (add 1)
#     - Use rolling to calculate accumulated growth factor (for loop)
#     - Apply to initial balance_eop to calculate balance_eop for each year (vectorized)

In [ ]:
# Once we have the evolution of balances, calculate taxable, non-taxable income, capital gains income, etc.